In [1]:
import pandas as pd
import numpy as np
import sklearn as sk 


In [37]:
#read in mbti_cleaned.csv and drop 'Unnamed: 0' column
df = pd.read_csv('mbti_cleaned.csv', encoding = "utf-8")
df = df.drop('Unnamed: 0', axis=1)

#select only entries with no null values
df = df[pd.notnull(df['clean_posts'])]
df = df[pd.notnull(df['posts'])]
df.head()

,type,posts,clean_posts
0,INFJ,enfp and intj moments https://www.youtube.com...,enfp intj moments sportscenter top ten plays p...
1,INFJ,What has been the most life-changing experienc...,lifechanging experience life
2,INFJ,http://www.youtube.com/watch?v=vXZeYwwRDw8 h...,repeat today
3,INFJ,May the PerC Experience immerse you.,may perc experience immerse
4,INFJ,The last thing my INFJ friend posted on his fa...,last thing infj friend posted facebook committ...


In [66]:
from sklearn.model_selection import train_test_split

corpus = df.clean_posts
labels = df.type

train_corpus, test_corpus, train_labels, test_labels  = sk.model_selection.train_test_split(corpus, labels, train_size=0.8, random_state=42)

In [67]:
from sklearn.feature_extraction.text import CountVectorizer
#toeknizing text with scikit-learn
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train_corpus)
X_train_counts.shape

(326400, 125951)

In [68]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(326400, 125951)

In [82]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, train_labels)

In [83]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted1 = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted1):
     print('%r => %s' % (doc, category))


'God is love' => INFP
'OpenGL on the GPU is fast' => INTP


In [62]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
 ])

In [71]:
text_clf.fit(train_corpus, train_labels)  

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [86]:
docs_new1 = test_corpus
X_new_counts1 = count_vect.transform(docs_new1)
X_new_tfidf1 = tfidf_transformer.transform(X_new_counts1)

predicted2 = text_clf.predict(X_new_tfidf1)
np.mean(predicted2 == test_labels)

AttributeError: lower not found

In [85]:
predicted2

array(['INFP', 'INFJ', 'INFP', ..., 'INFP', 'INFP', 'INFP'], 
      dtype='<U4')

In [78]:
from sklearn.linear_model import SGDClassifier
text_clf2 = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier())
 ])

text_clf2.fit(train_corpus, train_labels)


Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...   penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False))])

In [80]:
predicted3 = text_clf2.predict(docs_test)
np.mean(predicted3 == test_labels)

0.23417892156862746